Accuracy calculation function:

In [0]:
import numpy as np
from sklearn.cluster import KMeans

# Function to calculate accuracy (TAKEN FROM ORIGINAL CODE):
def calculate_accuracy(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size


Estimate accuracy score for K_Means:

In [2]:
from keras.datasets.mnist import load_data

(x_train, y_train), (x_test, y_test) = load_data()
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x = x.reshape((x.shape[0], -1))

k_means = KMeans(n_clusters=len(np.unique(y)))
y_est_k_means = k_means.fit_predict(x)

print(calculate_accuracy(y, y_est_k_means))

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
0.5323571428571429


Implement autoencoder, train it and estimate score for encoder + K_Means:

In [3]:
from keras.layers import Dense, Input
from keras.models import Model


input_layer = Input(shape=(784,), name='input') 

current = Dense(500, activation='relu')(input_layer)
current = Dense(500, activation='relu')(current)
current = Dense(2000, activation='relu')(current)

encoding_result = Dense(10)(current)

current = Dense(2000, activation='relu')(encoding_result)
current = Dense(500, activation='relu')(current)
current = Dense(500, activation='relu')(current)

current = Dense(784)(current)

decoding_result = current

autoencoder_full = Model(inputs=input_layer, outputs=decoding_result, name='autoencoder')
autoencoder_without_decoder = Model(inputs=input_layer, outputs=encoding_result, name='encoder')

autoencoder_full.compile(loss = 'mse', optimizer='adam')
autoencoder_full.fit(x, x, batch_size=256, epochs=20)

k_means_after_encoding = KMeans(n_clusters=10)
y_predicted = k_means_after_encoding.fit_predict(autoencoder_without_decoder.predict(x))

print()
print(calculate_accuracy(y, y_predicted))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
70000/70000 [==============================] - 44s 628us/step - loss: 2487.8858
Epoch 2/20
70000/70000 [==============================] - 43s 616us/step - loss: 1421.7480
Epoch 3/20
70000/70000 [==============================] - 43s 614us/step - loss: 1247.7960
Epoch 4/20
70000/70000 [==============================] - 43s 612us/step - loss: 1156.8260
Epoch 5/20
70000/70000 [==============================] - 42s 605us/step - loss: 1095.3879
Epoch 6/20
70000/70000 [==============================] - 42s 604us/step - loss: 1046.1051
Epoch 7/20
70000/70000 [==============================] - 44s 622us/step - loss: 1008.7352
Epoch 8/20
70000/70000 [==============================] - 44s 625us/step - loss: 979.4484
Epoch 9/20
70000/70000 [==============================] - 45s 636us/step - loss: 954.1710
Epoch 10/20
70000/70000 [==============================] - 45s 

DEC:

In [0]:
# Taken from https://github.com/XifengGuo/DEC-keras/blob/master/DEC.py:

from keras import backend
from keras.layers import Layer, InputSpec


class Clustering(Layer):

    def __init__(self, n_clusters, **kwargs):
        input_shape = (10,10)
        super(Clustering, self).__init__(**kwargs)
        self.n_clusters=10
        self.alpha = 1.0
        
    def build(self, input_shape):
        input_dimension = input_shape[1]
        self.input_spec = InputSpec(dtype=backend.floatx(), shape=(None, input_dimension))
        self.clusters = self.add_weight((self.n_clusters, input_dimension), initializer='glorot_uniform', name='clusters')
        super(Clustering, self).build(input_shape)

    def call(self, inputs):
        q = 1.0 / (1.0 + (backend.sum(backend.square(backend.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        return backend.transpose(backend.transpose(q) / backend.sum(q, axis=1)) 
        

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.n_clusters)

In [11]:
from keras.optimizers import SGD
from sklearn.cluster import KMeans


n_clusters = 10

clustering_layer = Clustering(n_clusters, name='clustering')(autoencoder_without_decoder.output)

model = Model(inputs=autoencoder_without_decoder.input, outputs=clustering_layer)

kmeans = KMeans(n_clusters=n_clusters, n_init=20)
kmeans.fit(autoencoder_without_decoder.predict(x))

model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])


model.compile(optimizer=SGD(0.01, 0.9), loss='kld')

index_array = np.arange(x.shape[0])
index = 0
maxiter = 5000
update_interval = 150
batch_size = 256

for iteration in range(maxiter):
  
    if iteration % update_interval == 0:
        q = model.predict(x, verbose=0)
        p = ((q ** 2 / q.sum(0)).T / (q ** 2 / q.sum(0)).sum(1)).T
        y_pred = q.argmax(1)
        
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0
    
print(calculate_accuracy(y, y_pred))

0.8376857142857143
